In [0]:
!pip install -q -U tensorflow>=1.8.0

In [13]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

x_train shape: (60000, 28, 28) y_train shape: (60000,)


In [14]:
x_train.shape

(60000, 28, 28)

In [0]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [16]:
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print(x_train.shape[0], 'train set')
print(x_valid.shape[0], 'validation set')
print(x_test.shape[0], 'test set')

x_train shape: (55000, 28, 28, 1) y_train shape: (55000, 10)
55000 train set
5000 validation set
10000 test set


In [17]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        8224      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
__________

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint(filepath='model_weights_best.h5', verbose = 1, save_best_only=True)
stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=10,
         validation_data=(x_valid, y_valid),
         callbacks=[checkpointer, stop])

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
54976/55000 [============================>.] - ETA: 0s - loss: 0.6031 - acc: 0.7796
Epoch 00001: val_loss improved from inf to 0.39689, saving model to model_weights_best.h5
55000/55000 [==============================] - 90s 2ms/sample - loss: 0.6029 - acc: 0.7796 - val_loss: 0.3969 - val_acc: 0.8592
Epoch 2/10
54976/55000 [============================>.] - ETA: 0s - loss: 0.4189 - acc: 0.8483
Epoch 00002: val_loss improved from 0.39689 to 0.31909, saving model to model_weights_best.h5
55000/55000 [==============================] - 91s 2ms/sample - loss: 0.4189 - acc: 0.8483 - val_loss: 0.3191 - val_acc: 0.8838
Epoch 3/10
54976/55000 [============================>.] - ETA: 0s - loss: 0.3745 - acc: 0.8638
Epoch 00003: val_loss improved from 0.31909 to 0.30448, saving model to model_weights_best.h5
55000/55000 [==============================] - 90s 2ms/sample - loss: 0.3745 - acc: 0.8638 - val_loss: 0.3045 - val_acc: 0.8882
Epoc

In [20]:
model.load_weights('model_weights_best.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.9106


In [0]:
from tensorflow.keras.models import load_model
model = load_model("model_weights_best.h5")
model_name = "model_weights_best.h5"